In [ ]:
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(6)
tf.config.threading.set_inter_op_parallelism_threads(6)


In [ ]:
import scipy.io
import numpy as np
import mne
import os
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:


from tensorflow import keras
from keras import layers
import time

dataset = []
labels = []
extension_dataset = []
flexion_dataset = []
rest_dataset = []
for filename in os.listdir('formated_data'):
    filepath = os.path.join('formated_data', filename)
    data = scipy.io.loadmat(filepath)
    data.pop('__header__')
    data.pop('__version__')
    data.pop('__globals__')
    

    data_arr = np.array(data['data'])
    
    data_arr = data_arr.T
    if 'EXTENSION' in filename:
        extension_dataset.append(data_arr)
    if 'FLEXION' in filename:
        flexion_dataset.append(data_arr)
    if 'REST' in filename:
        rest_dataset.append(data_arr)



In [ ]:
def read_data(dataset):
    for i in range(len(dataset)):
        #dataset[i] = np.resize(dataset[i],(7,16384))
        d = dataset[i].astype(float)
        dataset[i] = mne.filter.filter_data(d, l_freq=20, h_freq=450,sfreq=2000)
        print(d.shape)

    dataset = np.array(dataset)
    print(dataset.shape)
    return dataset


In [ ]:
extension_dataset[0].shape

In [ ]:

#chan_names = ['Fp1','AF7','AF3','AFz','F7','F5','F3','F1','Fz','FT7','FC5','FC3','FC1','T7','C5','C3','C1','Cz','TP7','CP5','CP3','CP1','CPz','P7','P5','P3','P1','Pz','PO7','PO3','POz','Fp2','AF4','AF8','F2','F4','F6','F8','FC2','FC4','FC6','FT8','C2','C4','C6','T8','CP2','CP4','CP6','TP8','P2','P4','P6','P8','PO4','PO8','O1','Oz','O2','Iz']
chan_names = ['ECR','FCR','ECU','FCU']
info = mne.create_info(ch_names=chan_names, ch_types=['emg']*4, sfreq=2000)
info.set_montage('standard_1020')
print(info)

In [ ]:
%%capture

extension_epochs = mne.EpochsArray(data=read_data(extension_dataset), info=info)
flexion_epochs = mne.EpochsArray(data=read_data(flexion_dataset), info=info)
rest_epochs = mne.EpochsArray(data=read_data(rest_dataset), info=info)
# extension_epochs1 = mne.EpochsArray(data=(extension_dataset), info=info)
# flexion_epochs1 = mne.EpochsArray(data=(flexion_dataset), info=info)
# rest_epochs1 = mne.EpochsArray(data=(rest_dataset), info=info)

In [ ]:
import matplotlib
from mne.time_frequency import tfr_morlet
%matplotlib inline
print(extension_epochs.get_data().shape)
print(flexion_epochs.get_data().shape)
print(rest_epochs.get_data().shape)
tmp = np.vstack((extension_epochs.get_data(),flexion_epochs.get_data(),rest_epochs.get_data()))
fs=2000

for i in range(10):
    print("________________________________ ",i+1)
    rand = random.randint(0,599)
    if(0 <= rand <= 199):
        print("extension")
    elif(200 <= rand <= 399):
        print("flexion")
    else:
        print("rest")
    
    d = tmp[rand]
    
    t_evoked = mne.EvokedArray(d, info, tmin=0)
    
    
    t_evoked.plot()
    
    print("________________________________ ")

In [ ]:
import numpy as np
import pywt

def min_max_scale_signal(signal, min_val=0, max_val=1):
    """
    Scale the signal to a new range [min_val, max_val].

    Parameters:
    - signal: Input signal, a numpy array.
    - min_val: Minimum value of the scaled range.
    - max_val: Maximum value of the scaled range.

    Returns:
    - Scaled signal.
    """
    min_signal = np.min(signal)
    max_signal = np.max(signal)
    scaled_signal = (signal - min_signal) / (max_signal - min_signal) * (max_val - min_val) + min_val
    return scaled_signal


In [ ]:
import scipy.io
import numpy as np
import mne
import os
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

ext_list = []
flex_list = []
rest_list = []

ext_list.append(extension_epochs.get_data())
flex_list.append(flexion_epochs.get_data())
rest_list.append(rest_epochs.get_data())
extension_labels = [0 for i in range(extension_epochs.get_data().shape[0])]
print(extension_labels)

flexion_labels = [1 for i in range(flexion_epochs.get_data().shape[0])]
print(flexion_labels)

rest_labels = [2 for i in range(rest_epochs.get_data().shape[0])]
print(rest_labels)

label_list = extension_labels + flexion_labels + rest_labels

extension_array = np.vstack(ext_list)
flexion_array = np.vstack(flex_list)
rest_array = np.vstack(rest_list)

extension_array = min_max_scale_signal(extension_array,-1,1)
flexion_array = min_max_scale_signal(flexion_array,-1,1)
rest_array = min_max_scale_signal(rest_array,-1,1)

label_array = np.hstack(label_list)

print(label_array.shape)
print(extension_array.shape,flexion_array.shape,rest_array.shape)

In [ ]:
import math
import pywt
import scipy.signal
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1).reshape(-1, 1)

def stddev(x):
    return np.std(x, axis=-1).reshape(-1, 1)

def peaktopeak(x):
    return np.ptp(x, axis=-1).reshape(-1, 1)

def variance(x):
    return np.var(x, axis=-1).reshape(-1, 1)

def mini(x):
    return np.min(x, axis=-1).reshape(-1, 1)

def maxi(x):
    return np.max(x, axis=-1).reshape(-1, 1)

def argmini(x):
    return np.argmin(x, axis=-1).reshape(-1, 1)

def argmaxi(x):
    return np.argmax(x, axis=-1).reshape(-1, 1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1)).reshape(-1, 1)

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1).reshape(-1, 1)

def skewness(x):
    return stats.skew(x, axis=-1).reshape(-1, 1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1).reshape(-1, 1)

def concat_features(x):
    features = np.concatenate(
        (
            peaktopeak(x),
            rms(x),
            abs_diff_signal(x),
            skewness(x),
            kurtosis(x),
            variance(x),
            mean(x),
            stddev(x)
        ),
        axis=1
    )
    return features


    
def featuresarray_load(data_array):
    features = []
    fs = 2000
    for d in data_array:
        
        beta_ftrs = concat_features(d)
        
        _,p=scipy.signal.welch(d, fs=fs,average='median')
        
        res = np.concatenate((beta_ftrs,p),axis=1)
        
        features.append(res)
    return np.array(features)

In [ ]:
f_ext = featuresarray_load(extension_array)
f_flex = featuresarray_load(flexion_array)
f_rest = featuresarray_load(rest_array)

np.save("ext_features_array.npy",allow_pickle=False,arr=f_ext)
np.save("flex_features_array.npy",allow_pickle=False,arr=f_flex)
np.save("rest_features_array.npy",allow_pickle=False,arr=f_rest)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import matplotlib.pyplot as plt

In [38]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, LayerNormalization, Reshape
from keras.optimizers import Adam,SGD
from keras.metrics import categorical_crossentropy
from keras.backend import clear_session
from matplotlib.ticker import MultipleLocator
# features_array = np.load(file="features_array3.npy")
# GLOBAL_SHAPE_LENGTH = features_array.shape[1]
# scaler = StandardScaler()
# features_array = scaler.fit_transform(features_array)

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, LayerNormalization, Reshape
from keras.optimizers import Adam,SGD
from keras.metrics import categorical_crossentropy
from keras.backend import clear_session

e_features_array = np.load(file="ext_features_array.npy")
f_features_array = np.load(file="flex_features_array.npy")
r_features_array = np.load(file="rest_features_array.npy")
features_array = np.concatenate((e_features_array,f_features_array,r_features_array))


print(features_array.shape)
GLOBAL_SHAPE_LENGTH = features_array.shape[2]
ncomp = features_array.shape[1]
print(GLOBAL_SHAPE_LENGTH)

scaler = MinMaxScaler((-1,1))


X_train, X_test, y_train, y_test = train_test_split(features_array, label_array, test_size=.2, random_state=42, shuffle=True, stratify=label_array)
features_array  = scaler.fit_transform(features_array.reshape(-1, features_array.shape[-1])).reshape(features_array.shape)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
lst = y_train.tolist()
print(lst.count(0),lst.count(1),lst.count(2),GLOBAL_SHAPE_LENGTH)

lst = y_test.tolist()
print(lst.count(0),lst.count(1),lst.count(2),GLOBAL_SHAPE_LENGTH)

scaler = MinMaxScaler((-1,1))

X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

import numpy as np
import matplotlib.pyplot as plt

# for _ in range(25):
#     sample_index_train = random.randint(0, len(X_train) - 1)
#     train_sample = X_train[sample_index_train]
#     train_label = y_train[sample_index_train]

#     match_indices = np.where(y_test == train_label)[0]
#     sample_index_test = random.choice(match_indices)
#     test_sample = X_test[sample_index_test]

#     time_axis = np.arange(train_sample.shape[1])  # Time points on the x-axis

#     plt.figure(figsize=(24, 6))

#     # Plot train sample
#     plt.subplot(1, 2, 1)
#     for channel in range(train_sample.shape[0]):
#         plt.plot(time_axis, train_sample[channel, :], label=f'Channel {channel+1}')
#     plt.title(f"Train Sample {sample_index_train}, Label {train_label}")
#     plt.xlabel("Time Point")
#     plt.ylabel("Signal Amplitude")
#     plt.ylim((-1, 1))
#     plt.legend()
#     plt.grid(True)

#     # Plot test sample
#     plt.subplot(1, 2, 2)
#     for channel in range(test_sample.shape[0]):
#         plt.plot(time_axis, test_sample[channel, :], label=f'Channel {channel+1}')
#     plt.title(f"Test Sample {sample_index_test}, Label {y_test[sample_index_test]}")
#     plt.xlabel("Time Point")
#     plt.ylabel("Signal Amplitude")
#     plt.ylim((-1, 1))
#     plt.legend()
#     plt.grid(True)

#     plt.show()



(600, 4, 137)
137
(480, 4, 137)
(120, 4, 137)
(480,)
(120,)
160 160 160 137
40 40 40 137


In [ ]:
import torch

In [ ]:
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch_size=32

In [31]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Define a custom dataset class for handling the EMG data
class EMGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)  # Convert data to torch tensors
        self.labels = torch.tensor(labels, dtype=torch.long)  # Convert labels to torch tensors

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Return a tuple of (data, label) for each index
        return self.data[index], self.labels[index]

# Assuming X_train and y_train are numpy arrays
# Create a custom dataset using the EMGDataset class
train_dataset = EMGDataset(X_train, y_train)
test_dataset = EMGDataset(X_test, y_test)

# Create DataLoader for the training and test sets
batch_size = 32  # Set an appropriate batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)


In [32]:
# Network Architecture
num_inputs = 4*137
num_hidden = 1000
num_outputs = 3

# Temporal Dynamics
num_steps = 25
beta = 0.95

In [34]:
from snntorch import surrogate
from torch import nn
import snntorch as snn

# Define the necessary parameters (ensure these match your model's requirements)
num_inputs = 4 * 137  # Flattened input size: 4 channels x 137 time steps
num_hidden = 1000     # Number of hidden units in the first fully connected layer
num_outputs = 3       # Number of output classes (assuming 3 classes in the target labels)
beta = 0.9            # Decay constant for the LIF neuron
num_steps = 137       # Number of time steps (should match the temporal dimension of your data)

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)  # Input: 548 (4*137), Output: 1000 hidden units
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)  # Output: 3 classes
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x):
        # Flatten the input outside the loop for all samples in the batch
        # Convert (batch_size, num_channels, num_timesteps) -> (batch_size, num_channels * num_timesteps)
        x = x.view(x.size(0), -1)  # Flatten the input to shape: (batch_size, 548)

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        
        # Record the final layer outputs
        spk2_rec = []
        mem2_rec = []

        # time-loop: Iterate through each time step
        for step in range(num_steps):
            # Pass through the first fully connected layer
            x = x.flatten(1)
            cur1 = self.fc1(x)  # Use the flattened input
            spk1, mem1 = self.lif1(cur1, mem1)

            # Pass through the second fully connected layer
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
          
            # Store in list
            spk2_rec.append(spk2)
            mem2_rec.append(mem2)

        # Stack the outputs over time to return the final tensor
        return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)  # Shape: (time_steps, batch_size, num_outputs)
        
# Load the network onto CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = Net().to(device)


In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, betas=(0.9, 0.999))

num_epochs = 15 # 60000 / 128 = 468
counter = 0
# Outer training loop
for epoch in range(num_epochs):
    train_batch = iter(train_loader)

    # Minibatch training loop
    for data, targets in train_batch:
        data = data.to(device)
        targets = targets.to(device)

        # forward pass
        net.train()
        spk_rec, _ = net(data)

        # initialize the loss & sum over time
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        loss_val = loss(spk_rec.sum(0), targets) # batch x num_out

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Print train/test loss/accuracy
        if counter % 10 == 0:
            print(f"Iteration: {counter} \t Train Loss: {loss_val.item()}")
        counter += 1

        if counter == 100:
            break

Iteration: 0 	 Train Loss: 9.703728675842285
Iteration: 10 	 Train Loss: 1.741128921508789
Iteration: 20 	 Train Loss: 0.9632488489151001
Iteration: 30 	 Train Loss: 1.1436772346496582
Iteration: 40 	 Train Loss: 1.0642805099487305
Iteration: 50 	 Train Loss: 0.6866699457168579
Iteration: 60 	 Train Loss: 0.4688836336135864
Iteration: 70 	 Train Loss: 0.55772465467453
Iteration: 80 	 Train Loss: 0.5328113436698914
Iteration: 90 	 Train Loss: 0.3433271050453186
Iteration: 100 	 Train Loss: 0.20605269074440002
Iteration: 110 	 Train Loss: 0.3877933919429779
Iteration: 120 	 Train Loss: 0.205989807844162
Iteration: 130 	 Train Loss: 0.34619802236557007
Iteration: 140 	 Train Loss: 0.41201362013816833
Iteration: 150 	 Train Loss: 0.44649457931518555
Iteration: 160 	 Train Loss: 0.41201990842819214
Iteration: 170 	 Train Loss: 0.30911222100257874
Iteration: 180 	 Train Loss: 0.2808725833892822
Iteration: 190 	 Train Loss: 0.38753166794776917
Iteration: 200 	 Train Loss: 0.30900537967681885


In [37]:
def measure_accuracy(model, dataloader):
  with torch.no_grad():
    model.eval()
    running_length = 0
    running_accuracy = 0

    for data, targets in iter(dataloader):
      data = data.to(device)
      targets = targets.to(device)

      # forward-pass
      spk_rec, _ = model(data)
      spike_count = spk_rec.sum(0) # batch x num_outputs
      _, max_spike = spike_count.max(1)

      # correct classes for one batch
      num_correct = (max_spike == targets).sum()

      # total accuracy
      running_length += len(targets)
      running_accuracy += num_correct
    
    accuracy = (running_accuracy / running_length)

    return accuracy.item()
print(f"Test set accuracy: {measure_accuracy(net, test_loader)}")

Test set accuracy: 0.6583333611488342


In [ ]:
import torch

# Calculate the memory size of the model parameters
model_size = sum(p.element_size() * p.numel() for p in net.parameters())
print(f"Model size: {model_size / 1024**2:.2f} MB")  # Convert to MB


In [ ]:
class Plasticity(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(X_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Box(low=-1, high=1,
                                                shape=(GLOBAL_SHAPE_LENGTH,ncomp),
                                                dtype=np.float32)
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):

        done = False
        reward = self.calculate_reward(action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()

        return obs
    
    def calculate_reward(self, action):
      if action == self.expected_action:
          reward = 1
          
      else:
          reward = 0

      return reward
    
    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]


        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            #print(f"Current dataset index: {self.dataset_idx}")
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs

In [ ]:
env = Plasticity()

In [ ]:
env.observation_space.sample()
env.observation_space.sample().shape

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Input
from tensorflow.keras.optimizers import Adam
tf.compat.v1.experimental.output_all_intermediates(True)

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n
print(states,actions)

In [ ]:
from keras.layers import PReLU, SpatialDropout1D, MaxPooling1D, GlobalMaxPooling1D, Layer, AveragePooling1D
from keras.regularizers import l1_l2
from tensorflow.keras.utils import register_keras_serializable
import keras.backend as K
# @keras.saving.register_keras_serializable()
# class LightweightLSTMAttentionLayer(Layer):
#     def __init__(self, **kwargs):
#         super(LightweightLSTMAttentionLayer, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.W = self.add_weight(name='att_weight',
#                                  shape=(input_shape[-1], 1),
#                                  initializer='glorot_uniform',
#                                  trainable=True)
#         self.b = self.add_weight(name='att_bias',
#                                  shape=(input_shape[1], 1),
#                                  initializer='zeros',
#                                  trainable=True)
#         super(LightweightLSTMAttentionLayer, self).build(input_shape)

#     def call(self, x):
#         # Attention mechanism
#         e = K.tanh(K.dot(x, self.W) + self.b)
#         e = K.squeeze(e, axis=-1)
#         alpha = K.softmax(e)
#         alpha = K.expand_dims(alpha, axis=-1)

#         # Apply the attention
#         context = x * alpha
#         context = K.sum(context, axis=1)
#         return context

#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], input_shape[-1])
    
def build_model(states, actions):
    clear_session()
    model = Sequential([
        Reshape((GLOBAL_SHAPE_LENGTH,ncomp,),input_shape=(1,ncomp,GLOBAL_SHAPE_LENGTH)),
        BatchNormalization(),

        Conv1D(64, kernel_size=7),
        PReLU(),
        BatchNormalization(),

        MaxPooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        Conv1D(128, kernel_size=5),
        BatchNormalization(),
        PReLU(),
        AveragePooling1D(pool_size=2),
        SpatialDropout1D(0.1),

        LSTM(128, activation='tanh', recurrent_regularizer=l1_l2(l1=0.01, l2=0.01),return_sequences=True),
        BatchNormalization(),
        GlobalMaxPooling1D(),
        BatchNormalization(),
        Dense(units=128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(units=actions, activation='linear')
    ])
    return model

In [ ]:
import tensorflow as tf
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from keras import __version__
model = build_model(states, actions)
model.summary()


In [ ]:
def build_agent(model, actions):
    policy = EpsGreedyQPolicy(eps=0.1)
    memory = SequentialMemory(limit=30000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-4)
    return dqn, policy

In [ ]:
dqn, policy = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=0.0055,decay=1e-4), metrics=['mse'])

In [ ]:
from rl.callbacks import Callback

class LossHistory(Callback):
    def __init__(self):
        self.losses = []

    def on_step_end(self, step, logs={}):
        self.losses.append(logs['metrics'][0])

class LossHistory2(Callback):
    def __init__(self):
        self.losses = []

    def on_step_end(self, step, logs={}):
        self.losses.append(logs['metrics'][1])

class RewardHistory(Callback):
    def on_train_begin(self, logs={}):
        self.rewards = []

    def on_episode_end(self, episode, logs={}):
        self.rewards.append(logs['episode_reward'])

class ExponentialDecayEpsilonCallback(Callback):
    def __init__(self, initial_epsilon, min_epsilon, decay_rate, decay_steps):
        self.epsilon = initial_epsilon
        self.min_epsilon = min_epsilon
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.step_count = 0

    def on_step_end(self, step, logs={}):
        self.step_count += 1
        self.epsilon = self.min_epsilon + (self.epsilon - self.min_epsilon) * np.exp(-self.step_count / self.decay_steps)
        self.model.policy.eps = max(self.epsilon, self.min_epsilon)
        #print(f"Step: {self.step_count}, New Epsilon: {self.epsilon}")



initial_epsilon = 1.0  
min_epsilon = 0.0 
decay_rate = 0.0001     
decay_steps = 100000    
epsilon_decay_cb = ExponentialDecayEpsilonCallback(initial_epsilon, min_epsilon, decay_rate, decay_steps)


loss_history = LossHistory()
loss_history2 = LossHistory2()
reward_history = RewardHistory()


dqn.fit(env, nb_steps=2000, callbacks=[loss_history,loss_history2,reward_history,epsilon_decay_cb], verbose=1)
dqn.policy.eps = 0
dqn.fit(env, nb_steps=200, callbacks=[loss_history,loss_history2,reward_history], verbose=1)
dqn.fit(env, nb_steps=200, callbacks=[loss_history,loss_history2,reward_history], verbose=1)

In [ ]:
class TestPhaseLogger(Callback):
    def __init__(self, env):
        
        self.env = env

    def on_action_end(self, action, logs={}):
       
        if not self.model.training and self.env.calculate_reward(action) > 0:
            print(f"Correct Action: {action}, Reward Gained")
        elif not self.model.training and self.env.calculate_reward(action) < 0:
            print(f"Wrong Action: {action}, Correct Action {self.env.y[self.env.dataset_idx-1]}, Penalty Incurred")
test_logger = TestPhaseLogger(env)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
from rl.policy import GreedyQPolicy
dqn.policy = GreedyQPolicy()

def dqn_eval(dqn_agent,d):
    attempts, correct = 0, 0
    labels = d[1]
    eenv = Plasticity(dataset=d, random=False)
    thing = 1
    y_predFull = []
    y_trueest = []
    total_reward = 0
    try:
        while True:
            if thing == 1:
                obs = eenv.reset()
                thing = 0
            done = False
            while not done:

                obs_reshaped = np.reshape(obs, (1,) + obs.shape)

                q_values = dqn_agent.compute_q_values(obs_reshaped)
                #print(q_values)

                action = np.argmax(q_values)
                
                #print('action: ', action, " - ", labels[attempts])
                y_predFull.append(action)
                y_trueest.append(labels[attempts])

                obs, rew, done, _ = eenv.step(action)
                if(rew == 2):
                    print(f"Correct Action: {action}, Reward Gained: {rew}")
                else:
                    print(f"Wrong Action: {action}, Correct Action {labels[attempts]}, Reward Gained: {rew}")
                total_reward += rew
                if done:
                    attempts += 1
    except StopIteration:
        print()
        print('Validation done...','total reward=',total_reward+1)

        y_predFull = np.array(y_predFull)
        y_trueest = np.array(y_trueest)

        cm = confusion_matrix(y_trueest, y_predFull)
        print(y_predFull)
        print(y_trueest)

        print("Confusion Matrix:\n", cm)

        report = classification_report(y_trueest, y_predFull,digits=4)
        print("Classification Report:\n", report)

        report = classification_report(y_trueest, y_predFull,output_dict=True)

        correct = sum(y_predFull == y_trueest)
        attempts = len(y_trueest)
        accuracy = (float(correct) / attempts)
        print('Validation done...')
        print('Accuracy: {:.2f}%'.format(accuracy*100))

        F1 = report['macro avg']['f1-score']
        precision = report['macro avg']['precision']
        recall = report['macro avg']['recall']

        print("F1 Score: {:.2f}%".format(F1*100))
        print("Precision: {:.2f}%".format(precision*100))
        print("Recall: {:.2f}%".format(recall*100))

    y_predFull = []
    y_trueest = []
    return accuracy, F1, precision, recall

dqn_eval(dqn,d=(X_train,y_train))
print('***************************************************************************************')
dqn_eval(dqn,d=(X_test,y_test))
print('***************************************************************************************')

def dqn_eval2(d,split=10):
     
    n_splits = split
    mean_rewards_per_fold = []
    acc_per_fold = []
    f1_mean, p_mean, r_mean = [], [], []
    # Loop over each fold
    
    skf = StratifiedKFold(n_splits=n_splits)
    train, test = d
    for train_index, test_index in skf.split(train,test):
        xtr, xte = train[train_index],train[test_index]
        ytr, yte = test[train_index], test[test_index]

        test_env = Plasticity(dataset = (xte, yte),random=True)
        
        scores = dqn.test(test_env, nb_episodes=10, visualize=False, verbose=0)
        acc, f1, p, r = dqn_eval(dqn,d=(xte, yte))
        
        mean_reward = np.mean(scores.history['episode_reward'])
        mean_rewards_per_fold.append(mean_reward)
        acc_per_fold.append(acc)
        f1_mean.append(f1)
        r_mean.append(r)
        p_mean.append(p)
   
    overall_mean_reward = np.mean(mean_rewards_per_fold)

    #print(acc_per_fold)
    
    f1_avg, p_avg, r_avg = np.mean(f1_mean), np.mean(p_mean), np.mean(r_mean)
    #print(mean_rewards_per_fold)
    print(f"Overall Mean Reward across all folds: {overall_mean_reward * 100} %")
    acc_avg = np.mean(acc_per_fold)
    print(f"Overall Mean Accuracy across all folds: {acc_avg * 100} %")
    #print(f1_mean)
    print(f"Overall Mean F1 across all folds: {f1_avg * 100} %")
    #print(p_mean)
    print(f"Overall Mean Precision across all folds: {p_avg * 100} %")
    #print(r_mean)
    print(f"Overall Mean Recall across all folds: {r_avg * 100} %")
#dqn_eval2(d=(X_train,y_train))

dqn_eval2(d=(features_array,label_array))
#dqn_eval(dqn,d=(features_array,label_array))

In [ ]:

dqn_eval(dqn,d=(X_test,y_test))

In [ ]:
scores = dqn.test(Plasticity(images_per_episode=1,random=True,dataset=(X_test,y_test)), callbacks=[test_logger],nb_episodes=30, visualize=False, action_repetition=1, verbose=1)
print(np.mean(scores.history['episode_reward'])*100,'%')
plt.plot(scores.history['episode_reward'])
plt.title('Testing Rewards per Episode')
plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')
plt.show()
print(np.mean(scores.history['episode_reward'])*100,'%')
print(label_array,label_array.shape)

dqn_eval(dqn,d=(features_array,label_array))
def moving_average(data, window_size):
    return [np.mean(data[i:i+window_size]) for i in range(len(data) - window_size + 1)]
losses = loss_history.losses
losses2 = loss_history2.losses
smoothed_losses = moving_average(losses, window_size=50)
print(smoothed_losses)
plt.plot(smoothed_losses)
plt.title('Smoothed Training Loss per Step')
plt.xlabel('Step')
plt.xlim(left=0)
plt.ylabel('Smoothed Loss')
plt.grid(True)
plt.show()

smoothed_losses2 = moving_average(losses2, window_size=70)
print(smoothed_losses2)
plt.plot(smoothed_losses2)
plt.title('Smoothed Training Loss per Step')
plt.xlabel('Step')
plt.xlim(left=0)
plt.ylabel('Smoothed Loss')
plt.grid(True)
plt.show()

plt.plot(moving_average(reward_history.rewards, window_size=70))
plt.title('Training Rewards per Episode')
plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')
plt.show()

print("___________________________________________________________________________________________________________________________")




